# Kryptosystem Classic McEliece

In [ ]:
#troche o kodach korekcyjnych

### Kody Goppa
Są to kody korekcyjne stosowane wykorzystywane przez kryptosystem McElisse do tworzenia macierzy Generatora. Kody te zakładają że wiadomość szyfrowana ma postać:

$w_x(a) = \sum_{i=1}^{k}*a^{i-1}$

Kod Goppa wykorzystuje ciała skońćzone Galios $\mathbb{F}_{2^n}$ o współcynnikach binarnych.

poniżej znajdują się Struktua realizująca ciało skończone Galios

numpy.ndarray

In [2]:
import numpy as np

class Zn:
    def __init__(self, num, n) -> None:
        self.n = n
        self.num = num % n
    
    def __add__(self, other):
        if isinstance(other, int) or isinstance(other, np.intc): other = Zn(other, self.n)
        return Zn((self.num + other.num), self.n)
    
    def __sub__(self, other):
        if isinstance(other, int): other = Zn(other, self.n)
        return Zn((self.num - other.num + self.n), self.n)
    
    def __mul__(self, other):
        if isinstance(other, int): other = Zn(other, self.n)
        return Zn((self.num * other.num), self.n)
    
    def __str__(self) -> str:
        return f"{self.num}"

    def __repr__(self) -> str:
        return f"Z{self.n}.{self.num}"
    
    def prim_root(self):
        all_needed = {1}
        for m in range(2, self.n):
            if gcd(m,self.n) == 1:
                all_needed.add(m)
        
        for m in range(2,self.n):
            if gcd(m,self.n) != 1:
                continue
            start_num = 1
            numbers = {1}

            next_num = m

            while next_num != 1:
                numbers.add(next_num)
                next_num *= m
                next_num %= self.n

            if all_needed == numbers:
                return m
        return 0

class Poly:
    def __init__(self, n:int, T:np.ndarray):
        self.n = n
        if len(T) > n:
            for i in range(n, len(T)):
                T[i % n] += T[i]
        else:
            T = np.pad(T, (0, n - len(T)), 'constant')
        
        self.T = T[:n] % 2

    def val(self, x:int):
        ans = Zn(0, self.n)
        for power, coeff in enumerate(self.T):
            ans = ans + coeff * (x ** power)
        return ans.num

    def __str__(self):
        terms = []
        for power, coeff in enumerate(self.T):
            if coeff != 0:
                if power == 0:
                    terms.append("1")
                elif power == 1:
                    terms.append("x")
                else:
                    terms.append(f"x^{power}")
        
        if not terms:
            return "0"
        else:
            return " + ".join(terms)
    
    def __repr__(self) -> str:
        return self.__str__()

p = Poly(5, np.array([1, 1, 1, 1,0,1,1,0,1,1,0,0,1]))
print(p, p.val(1))

wspolczynniki = np.array([1, 1, 0, 1])
wielomian = Poly(2, wspolczynniki)
wielomian



x^4 1


1

## Generowanie kluczy

In [4]:
!pip install mathutils

     -------------------------------------- 245.4/245.4 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for mathutils: filename=mathutils-3.3.0-cp310-cp310-win_amd64.whl size=100523 sha256=30a7081305c35ee65cd3c07b7047544450aa503d04c15786f092b538cffe49e6
  Stored in directory: c:\users\ivanz\appdata\local\pip\cache\wheels\b7\6b\42\1ff5f16ba6189b674b71b6b5d161809ae733fac6e3a3365ed7
Successfully built mathutils


In [7]:
from mathutils import *
from sympy.polys.galoistools import gf_irreducible, gf_irreducible_p
from sympy import lcm, ZZ
from sympy.abc import x, alpha
from sympy import Matrix
import logging

log = logging.getLogger("goppacodegenerator")


class GoppaCodeGenerator:

    def __init__(self, m, n, t):
        self.m = m
        self.n = n
        self.t = t
        self.q = 2
        log.info(f"GoppaCodeGenerator(m={self.m},n={self.n},t={self.t},q={self.q},q^m={self.q ** self.m}) initiated")

    def gen(self):
        print(alpha)
        irr_poly = Poly(alpha ** self.m + alpha + 1, alpha).set_domain(GF(self.q))
        if is_irreducible_poly(irr_poly, self.q):
            ring = power_dict(self.q ** self.m, irr_poly, self.q)
        else:
            ring = []
        log.info("irr(q_size: {}): {}".format(len(ring), irr_poly))
        while len(ring) < self.q ** self.m - 1:
            irr_poly = irreducible_poly(self.m, self.q, alpha)
            ring = power_dict(self.q ** self.m, irr_poly, self.q)
            log.info("irr(q_size: {}): {}".format(len(ring), irr_poly))

        log.debug(f"ring={ring}")
        g_poly = Poly(1, x)

        roots_num = max(0, self.q ** self.m - self.n - self.t)

        #g_roots = np.random.choice(range(self.q ** self.m - 1), roots_num, replace=False)
        g_roots = set()
        g_non_roots = list(set(range(self.q ** self.m - 1)) - set(g_roots))

        log.debug(f"g_roots({len(g_roots)})={g_roots}")
        log.debug(f"g_non_roots({len(g_non_roots)})={g_non_roots}")

        for i in g_roots:
            g_poly = (g_poly * Poly(x + alpha ** i, x)).trunc(self.q)

        if g_poly.degree() < self.t:
            small_irr = None
            for i in range(100):
                small_irr = irreducible_poly_ext_candidate(self.t - g_poly.degree(), irr_poly, self.q, x, non_roots=g_non_roots)
                log.debug(f"irr_part_of_g={small_irr}")
                if small_irr.eval(0).is_zero or small_irr.eval(1).is_zero:
                    log.debug(f'roots in trivial case 0:{small_irr.eval(0)} 1:{small_irr.eval(1)}')
                    continue
                first_root = first_alpha_power_root(small_irr, irr_poly, self.q)
                if first_root > 0:
                    log.debug(f"alpha^{first_root} is a root of g(x)={small_irr}")
                    continue
                break
            else:
                raise Exception("irr poly not found")
            g_poly = (g_poly * small_irr).trunc(self.q)

        g_poly = reduce_to_alpha_power(g_poly, irr_poly, ring, self.q)
        log.info(f"g(x)={g_poly}")
        coeffs = g_poly.all_coeffs()

        first_root = first_alpha_power_root(g_poly, irr_poly, self.q, elements_to_check=g_non_roots)
        if first_root > 0:
            raise Exception(f"alpha^{first_root} is a root of g(x)={g_poly}")

        C = Matrix(self.t, self.t, lambda i, j: coeffs[j - i] if 0 <= j - i < self.t else 0)
        log.debug(f"C={C}")
        X = Matrix(self.t, self.n, lambda i, j: (alpha ** ((j * (self.t - i - 1)) % self.n)))
        log.debug(f"X={X}")
        Y = Matrix(self.n, self.n,
                   lambda i, j: get_alpha_power(g_poly.eval(alpha ** g_non_roots[i]), irr_poly, ring, self.q, neg=True)
                   if i == j else 0)
        log.debug(f"Y={Y}")
        H = C * X * Y
        H = Matrix(self.t, self.n, lambda i, j: get_alpha_power(H[i, j], irr_poly, ring, self.q))
        log.debug(f"H=\n{H}")
        H_bin = np.array(
            [np.column_stack([get_binary_from_alpha(e, irr_poly, self.q) for e in line]) for line in
             H.tolist()]).astype(GF2)
        H_bin = GF2Matrix.from_list(H_bin.reshape(-1, H.shape[1]))
        log.info(f"H_bin=\n{H_bin}")
        H_nullspace, nullity = H_bin.nullspace()
        log.debug(f"H_nullspace({nullity})=\n{H_nullspace}")
        G = GF2Matrix(H_nullspace.T()[:nullity])
        log.info(f"G=\n{G}")
        log.debug(f"G*H^T=\n{G * H_bin.T()}")
        return G, H_bin, g_poly, irr_poly

In [8]:
m = 12
n = 10
t = 1

G, H, g_poly, irr_poly = GoppaCodeGenerator(m, n, t).gen()
g_poly = np.array([(Poly(e, alpha) % irr_poly).trunc(2).all_coeffs()[::-1] for e in
                        self.g_poly.all_coeffs()[::-1]])
irr_poly = np.array(irr_poly.all_coeffs()[::-1])
k = G.arr.shape[0]
P = GF2Matrix.from_list(random_perm_matrix(n))
P_inv = P.inv()
S = GF2Matrix.from_list(random_inv_matrix(k))
S_inv = S.inv()
Gp = S * G * P

alpha


TypeError: object of type 'Symbol' has no len()

# Enkapsulacja kluczy

# Dekapsulacja klucza

In [10]:
from mceliece.mathutils import *
from goppa.goppacodegenerator import *
import logging
import timeit

log = logging.getLogger("ntrucipher")


class McElieceCipher:

    def __init__(self, m, n, t):
        self.m = m
        self.n = n
        self.t = t
        self.q = 2
        log.info(f"McEliece(m={self.m},n={self.n},t={self.t},q={self.q},q^m={self.q ** self.m}) initiated")
        self.G = None
        self.H = None
        self.k = None
        self.P = None
        self.P_inv = None
        self.S = None
        self.S_inv = None
        self.Gp = None
        self.g_poly = None
        self.irr_poly = None

    def generate_random_keys(self):
        self.G, self.H, self.g_poly, self.irr_poly = GoppaCodeGenerator(self.m, self.n, self.t).gen()
        self.g_poly = np.array([(Poly(e, alpha) % self.irr_poly).trunc(2).all_coeffs()[::-1] for e in
                                self.g_poly.all_coeffs()[::-1]])
        self.irr_poly = np.array(self.irr_poly.all_coeffs()[::-1])
        self.k = self.G.arr.shape[0]
        self.P = GF2Matrix.from_list(random_perm_matrix(self.n))
        self.P_inv = self.P.inv()
        self.S = GF2Matrix.from_list(random_inv_matrix(self.k))
        self.S_inv = self.S.inv()
        self.Gp = self.S * self.G * self.P

    def encrypt(self, msg_arr):
        if len(msg_arr) != self.Gp.shape[0]:
            raise Exception(f"Wrong message length. Should be {self.Gp.shape[0]} bits.")
        log.debug(f"msg: {msg_arr}")
        Cp = GF2Matrix.from_list(msg_arr) * GF2Matrix.from_list(self.Gp)
        log.debug(f"C': {Cp}")
        bits_to_flip = np.random.choice(len(Cp), size=self.t, replace=False)
        log.debug(f"bits_to_flip: {bits_to_flip}")
        for b in bits_to_flip:
            Cp[b] = Cp[b].flip()
        log.debug(f"C': {Cp}")
        return Cp

    def repair_errors(self, msg_arr, syndrome):
        if type(self.irr_poly) != GF2Poly:
            self.irr_poly = GF2Poly.from_numpy(self.irr_poly)
        ring = GF2mRing(self.m, self.irr_poly)
        if type(self.g_poly) != GF2mPoly:
            self.g_poly = GF2mPoly.from_list(
            [GF2m(GF2Poly.from_list([int(e) for e in coeff]), ring) for coeff in self.g_poly])
        log.debug(f'irr_poly:{self.irr_poly}')
        log.debug(f'g_poly:{self.g_poly}')

        S_poly = GF2mPoly.from_list(
            [GF2m(GF2Poly.from_list([int(e) for e in syndrome[i * self.m:(i + 1) * self.m].flat]), ring) for i in
             range(len(syndrome) // self.m)])
        log.debug(f'S_poly={S_poly}')
        S_inv_poly = S_poly.inv_mod(self.g_poly)
        log.debug(f'S_inv_poly={S_inv_poly}')
        log.debug(f'S_poly*S_inv_poly (mod g_poly: {self.g_poly})={(S_poly*S_inv_poly)%self.g_poly}')

        if S_inv_poly.degree() == 1 and S_inv_poly[1].n.degree() == 0 and S_inv_poly[1].n.poly.coeffs()[0] == 1:
            tau_poly = S_inv_poly
        else:
            g0, g1 = self.g_poly.split()

            log.debug(f"g0:{g0};g1:{g1}")
            log.debug(f"g0^2 + z*g1^2 :{g0**2 + GF2mPoly.x(ring)*g1**2}")
            log.debug(f'g1_inv:{g1.inv_mod(self.g_poly)}')

            w = g0 * g1.inv_mod(self.g_poly)
            log.debug(f"w:{w}")

            H_poly = S_inv_poly + GF2mPoly.from_list(
                [GF2m(GF2Poly.from_list([0]), ring), GF2m(GF2Poly.from_list([1]), ring)])
            log.debug(f'H_poly={H_poly}')

            H0, H1 = H_poly.split()
            log.debug(f'H0={H0};H1={H1}')

            R = H0 + w * H1

            log.debug(f'R:{R}')
            log.debug(f'R^2 mod g:{(R**2)%self.g_poly}')

            b, _, a = ext_euclid_poly_alt(R, self.g_poly, ring, self.t)

            log.debug(f'a:{a};b:{b}')
            log.debug(f'a**2:{a**2};b**2:{b**2};z*b**2:{GF2mPoly.x(ring)*b**2}')
            log.debug(f'b*R mod g:{(b*R)%self.g_poly}')

            tau_poly = (a ** 2 + GF2mPoly.x(ring) * b ** 2)

        log.debug(f'tau_poly={tau_poly}')
        test_elem = ring.one()
        for i in range(len(msg_arr)):
            value = tau_poly.eval(test_elem)
            log.debug(f't(alpha^{i})={value}')
            if value == 0:
                msg_arr[i] = msg_arr[i].flip()
                log.info(f"REPAIRED ERROR ON {i}th POSITION")
            test_elem = test_elem * ring.alpha()

        return msg_arr

    def decode(self, msg_arr):
        if type(msg_arr) != GF2Matrix:
            msg_arr = GF2Matrix.from_list(msg_arr)
        log.debug(f'msg_len:{len(msg_arr)}')
        syndrome = msg_arr * GF2Matrix.from_list(self.H.T)
        log.info(f'syndrome:\n{syndrome}')
        if not all(syndrome.arr == 0):
            msg_arr = self.repair_errors(msg_arr, syndrome)

        D = GF2Matrix.from_list(np.append(self.G.T, msg_arr.arr.reshape(len(msg_arr), 1), axis=1))
        log.debug(f'G^T|c=')
        D_rref = rref(D, steps=self.G.shape[0])
        log.debug(f'I|m=\n{D_rref}')

        return GF2Matrix.from_list(D_rref[:self.G.shape[0], self.G.shape[0]:].flatten())

    def decrypt(self, msg_arr):
        if len(msg_arr) != self.H.shape[1]:
            raise Exception(f"Wrong message length. Should be {self.H.shape[1]} bits.")
        log.debug(f"msg: {msg_arr}")
        Cp = GF2Matrix.from_list(msg_arr) * GF2Matrix.from_list(self.P_inv)
        log.debug(f"C': {Cp}")
        Mp = self.decode(Cp)
        log.debug(f"m': {Mp}")
        M = Mp * GF2Matrix.from_list(self.S_inv)
        log.debug(f"msg: {M}")
        return M.to_numpy()

ModuleNotFoundError: No module named 'mceliece'